# 2025 Fantasy Football Payouts
This notebook automatically fetches league data from ESPN and calculates payouts.

**What's Automatic:**
- Number of teams
- Number of regular season weeks
- Playoff results (1st, 2nd, 3rd place)
- Weekly scores

**What You Configure:**
- Buy-in amount
- Payout percentages and structure

---
# ⚠️ USER INPUT REQUIRED ⚠️

**Configure your league's payout structure below.**

Everything else (teams, weeks, playoff results) is fetched automatically from ESPN!

In [ ]:
# ============================================================================
# LEAGUE PAYOUT CONFIGURATION (Update these for your league)
# ============================================================================

BUY_IN = 50                        # Buy-in amount per team
SHARE_FOR_WEEKLY_HIGHS = 0.20      # 20% of pot goes to weekly highs
FIRST_PLACE_SHARE = 0.75           # 75% of winners pot
SECOND_PLACE_SHARE = 0.25          # 25% of winners pot
THIRD_PLACE_PAYOUT = 50            # Fixed payout for 3rd place

# ============================================================================

print("League Payout Configuration:")
print(f"  Buy-in per team: ${BUY_IN}")
print(f"  Weekly highs share: {SHARE_FOR_WEEKLY_HIGHS*100}%")
print(f"  1st place share: {FIRST_PLACE_SHARE*100}%")
print(f"  2nd place share: {SECOND_PLACE_SHARE*100}%")
print(f"  3rd place payout: ${THIRD_PLACE_PAYOUT}")

---
## Fetch League Data from ESPN API

In [ ]:
import requests
import pandas as pd
import os
import dotenv
import json

dotenv.load_dotenv()

year = 2025
league_id = os.getenv('league_id')
espn_s2 = os.getenv('espn_s2')
swid = os.getenv('swid')

cookies = {'espn_s2': espn_s2, 'SWID': swid}
url = f"https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/{year}/segments/0/leagues/{league_id}"

print("✓ Loaded credentials")

In [ ]:
# Fetch all required data in one optimized call
params = {"view": ["mTeam", "mMatchup", "mMatchupScore", "mStandings"]}
headers = {
    "X-Fantasy-Filter": json.dumps({"schedule": {"filterMatchupPeriodIds": {"value": list(range(1, 18))}}})
}

try:
    response = requests.get(url, cookies=cookies, params=params, headers=headers)
    response.raise_for_status()
    league_data = response.json()
    print("✓ Successfully fetched all league data")
except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching league data: {e}")
    raise

In [ ]:
# Extract league settings
settings = league_data.get('settings', {})
NUM_TEAMS = len(league_data.get('teams', []))
NUM_REG_SEASON_WEEKS = (
    settings.get('scheduleSettings', {}).get('matchupPeriodCount') or
    settings.get('regularSeasonMatchupPeriodCount') or
    settings.get('scheduleSettings', {}).get('regularSeasonMatchupPeriodCount') or
    14  # Default fallback
)

# Calculate pot
TOTAL_POT = NUM_TEAMS * BUY_IN
WEEKLY_HIGHS_POT = TOTAL_POT * SHARE_FOR_WEEKLY_HIGHS
WINNERS_POT = TOTAL_POT - WEEKLY_HIGHS_POT

print("\n" + "="*60)
print("LEAGUE SETTINGS (from ESPN API)")
print("="*60)
print(f"  League ID: {league_id}")
print(f"  Season: {year}")
print(f"  Number of Teams: {NUM_TEAMS}")
print(f"  Regular Season Weeks: {NUM_REG_SEASON_WEEKS}")
print(f"\nCalculated Pot:")
print(f"  Total: ${TOTAL_POT} ({NUM_TEAMS} teams × ${BUY_IN})")
print(f"  Weekly Highs: ${WEEKLY_HIGHS_POT}")
print(f"  Winners: ${WINNERS_POT}")
print("="*60)

In [ ]:
# Build optimized team and member lookups
members = {m['id']: f"{m.get('firstName', '')} {m.get('lastName', '')}".strip() 
           for m in league_data.get('members', [])}

teams = {}
for team in league_data.get('teams', []):
    owner_id = team.get('owners', [None])[0]
    teams[team['id']] = {
        'name': team.get('name', 'Unknown Team'),
        'owner': members.get(owner_id, 'Unknown Owner')
    }

# Create reverse mapping for manual entry fallback
team_to_owner = {info['name']: info['owner'] for info in teams.values()}

print(f"✓ Processed {len(teams)} teams and {len(members)} members")

In [ ]:
# Extract all scores into DataFrame
scores_data = []
for matchup in league_data.get('schedule', []):
    week = matchup.get('matchupPeriodId')
    for side in ['home', 'away']:
        if team := matchup.get(side):
            team_info = teams.get(team.get('teamId'), {})
            scores_data.append({
                'Week': week,
                'Team Name': team_info.get('name', 'Unknown'),
                'Owner': team_info.get('owner', 'Unknown'),
                'Score': team.get('totalPoints', 0)
            })

df = pd.DataFrame(scores_data)
print(f"✓ Processed {len(df)} records from weeks {sorted(df['Week'].unique())}")

# Display teams reference
teams_ref_df = pd.DataFrame([{'Team Name': info['name'], 'Owner': info['owner']} 
                              for info in teams.values()]).sort_values('Team Name')

print("\n" + "="*60)
print("ALL TEAMS IN YOUR LEAGUE")
print("="*60)
print(teams_ref_df.to_string(index=False))
print("="*60)

## Fetch Playoff Results

In [ ]:
# Attempt to automatically detect playoff winners
print("\nAttempting to fetch playoff results...")

try:
    playoff_results = []
    for team in league_data.get('teams', []):
        team_info = teams.get(team['id'], {})
        final_rank = team.get('rankCalculatedFinal') or team.get('rankFinal') or team.get('playoffSeed', 0)
        playoff_results.append({
            'Team Name': team_info.get('name', 'Unknown'),
            'Owner': team_info.get('owner', 'Unknown'),
            'Final Rank': final_rank,
            'Playoff Seed': team.get('playoffSeed', 0)
        })
    
    playoff_df = pd.DataFrame(playoff_results).sort_values('Final Rank')
    
    print("\n" + "="*60)
    print("PLAYOFF STANDINGS")
    print("="*60)
    print(playoff_df.to_string(index=False))
    print("="*60)
    
    # Auto-detect top 3 if available
    ranks = playoff_df[playoff_df['Final Rank'].isin([1, 2, 3])]
    
    if len(ranks) == 3:
        winners = ranks.set_index('Final Rank')[['Team Name', 'Owner']].to_dict('index')
        FIRST_PLACE_TEAM, FIRST_PLACE_WINNER = winners[1]['Team Name'], winners[1]['Owner']
        SECOND_PLACE_TEAM, SECOND_PLACE_WINNER = winners[2]['Team Name'], winners[2]['Owner']
        THIRD_PLACE_TEAM, THIRD_PLACE_WINNER = winners[3]['Team Name'], winners[3]['Owner']
        
        print("\n✅ PLAYOFF RESULTS AUTOMATICALLY DETECTED:")
        print(f"  🥇 1st Place: {FIRST_PLACE_TEAM} ({FIRST_PLACE_WINNER})")
        print(f"  🥈 2nd Place: {SECOND_PLACE_TEAM} ({SECOND_PLACE_WINNER})")
        print(f"  🥉 3rd Place: {THIRD_PLACE_TEAM} ({THIRD_PLACE_WINNER})")
        AUTO_DETECTED = True
    else:
        print("\n⚠️  Playoffs incomplete. Please enter results manually in next cell.")
        AUTO_DETECTED = False
        
except Exception as e:
    print(f"\n⚠️  Error: {e}. Please enter results manually.")
    AUTO_DETECTED = False

---
# ⚠️ MANUAL INPUT (Only if needed) ⚠️

In [ ]:
if not AUTO_DETECTED:
    print("Enter team names manually:\n")
    
    FIRST_PLACE_TEAM = 'Make Gronk Great Again'   # ← Replace
    SECOND_PLACE_TEAM = 'El Diablo'  # ← Replace
    THIRD_PLACE_TEAM = 'Caleb\'s B*tch'   # ← Replace
    
    FIRST_PLACE_WINNER = team_to_owner.get(FIRST_PLACE_TEAM)
    SECOND_PLACE_WINNER = team_to_owner.get(SECOND_PLACE_TEAM)
    THIRD_PLACE_WINNER = team_to_owner.get(THIRD_PLACE_TEAM)
    
    if all([FIRST_PLACE_WINNER, SECOND_PLACE_WINNER, THIRD_PLACE_WINNER]):
        print("✓ Manual entry validated:")
        print(f"  🥇 1st: {FIRST_PLACE_TEAM} ({FIRST_PLACE_WINNER})")
        print(f"  🥈 2nd: {SECOND_PLACE_TEAM} ({SECOND_PLACE_WINNER})")
        print(f"  🥉 3rd: {THIRD_PLACE_TEAM} ({THIRD_PLACE_WINNER})")
    else:
        print("⚠️  Invalid team names. Check spelling against table above.")
else:
    print("✓ Using auto-detected results.")

---
# 📊 Payout Calculations

In [ ]:
# Calculate payouts
FIRST_PLACE_PAYOUT = WINNERS_POT * FIRST_PLACE_SHARE
SECOND_PLACE_PAYOUT = WINNERS_POT * SECOND_PLACE_SHARE

print("\n" + "="*60)
print("PAYOUT STRUCTURE")
print("="*60)
print(f"Total Pot: ${TOTAL_POT} ({NUM_TEAMS} teams × ${BUY_IN})")
print(f"\nDistribution:")
print(f"  Weekly Highs ({SHARE_FOR_WEEKLY_HIGHS*100}%): ${WEEKLY_HIGHS_POT}")
print(f"  Winners ({(1-SHARE_FOR_WEEKLY_HIGHS)*100}%): ${WINNERS_POT}")
print(f"\nWinner Payouts:")
print(f"  🥇 1st ({FIRST_PLACE_SHARE*100}%): ${FIRST_PLACE_PAYOUT}")
print(f"  🥈 2nd ({SECOND_PLACE_SHARE*100}%): ${SECOND_PLACE_PAYOUT}")
print(f"  🥉 3rd (fixed): ${THIRD_PLACE_PAYOUT}")
print(f"\nTotal Winners: ${FIRST_PLACE_PAYOUT + SECOND_PLACE_PAYOUT + THIRD_PLACE_PAYOUT}")
print("="*60)

In [ ]:
# Calculate weekly high scores (optimized with pandas)
regular_season = df[df['Week'] <= NUM_REG_SEASON_WEEKS]
weekly_highs = regular_season.loc[regular_season.groupby('Week')['Score'].idxmax()].sort_values('Week')

# Calculate payouts
weekly_payout_per_win = WEEKLY_HIGHS_POT / NUM_REG_SEASON_WEEKS
weekly_summary = (weekly_highs.groupby('Owner', as_index=False).size()
                  .rename(columns={'size': 'Weekly High Count'}))
weekly_summary['Weekly High Payout'] = weekly_summary['Weekly High Count'] * weekly_payout_per_win

print("\nWeekly High Scores:")
print(weekly_highs[['Week', 'Team Name', 'Owner', 'Score']].to_string(index=False))
print(f"\nWeekly High Payouts (${weekly_payout_per_win:.2f} per week):")
print(weekly_summary.to_string(index=False))

In [ ]:
# Build final payout summary (optimized with merge)
# Create base DataFrame with all unique owners
payout_df = df[['Owner', 'Team Name']].drop_duplicates('Owner').copy()

# Add placement payouts
placement_map = {
    FIRST_PLACE_WINNER: (1, FIRST_PLACE_PAYOUT),
    SECOND_PLACE_WINNER: (2, SECOND_PLACE_PAYOUT),
    THIRD_PLACE_WINNER: (3, THIRD_PLACE_PAYOUT)
}
payout_df['Place'] = payout_df['Owner'].map(lambda x: placement_map.get(x, (None, 0))[0])
payout_df['Place Payout'] = payout_df['Owner'].map(lambda x: placement_map.get(x, (None, 0))[1])

# Merge weekly high payouts
payout_df = payout_df.merge(weekly_summary, on='Owner', how='left')
payout_df['Weekly High Count'] = payout_df['Weekly High Count'].fillna(0).astype(int)
payout_df['Weekly High Payout'] = payout_df['Weekly High Payout'].fillna(0)

# Calculate total
payout_df['Total Payout'] = payout_df['Place Payout'] + payout_df['Weekly High Payout']
payout_df['Place'] = payout_df['Place'].fillna('')

# Sort by payout
payout_df = payout_df.sort_values('Total Payout', ascending=False)

print("\n" + "="*80)
print("FINAL PAYOUT SUMMARY")
print("="*80)
print(payout_df.to_string(index=False))
print("="*80)
print(f"\nTotal Paid Out: ${payout_df['Total Payout'].sum():.2f}")
print(f"Expected Total: ${TOTAL_POT:.2f}")
print(f"Difference: ${abs(payout_df['Total Payout'].sum() - TOTAL_POT):.2f}")

## Export Auditable Results

In [ ]:
# Export to multi-tab Excel file for Google Sheets
output_file = f'outputs/fantasy_football_payouts_{year}.xlsx'

# Create parameters summary DataFrame
params_df = pd.DataFrame({
    'Parameter': ['Teams', 'Buy In', 'Weekly Highs %', '1st Place %', '2nd Place %', '3rd Payout', 
                  'Reg Season Weeks', '', 'Total Pot', 'Winners Pot', 'Weekly Highs Pot', '', 
                  '1st Payout', '2nd Payout', '3rd Payout', 'Total Winners', '', 'Check'],
    'Value': [NUM_TEAMS, BUY_IN, f"{SHARE_FOR_WEEKLY_HIGHS*100}%", f"{FIRST_PLACE_SHARE*100}%", 
              f"{SECOND_PLACE_SHARE*100}%", THIRD_PLACE_PAYOUT, NUM_REG_SEASON_WEEKS, '', 
              TOTAL_POT, WINNERS_POT, WEEKLY_HIGHS_POT, '', FIRST_PLACE_PAYOUT, SECOND_PLACE_PAYOUT, 
              THIRD_PLACE_PAYOUT, FIRST_PLACE_PAYOUT + SECOND_PLACE_PAYOUT + THIRD_PLACE_PAYOUT, '',
              'Yes' if abs((WINNERS_POT + WEEKLY_HIGHS_POT) - TOTAL_POT) < 0.01 else 'No']
})

try:
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Tab 1: Final Payout Summary
        payout_df.to_excel(writer, sheet_name='Final Payouts', index=False)
        
        # Tab 2: Weekly High Scores
        weekly_highs[['Week', 'Team Name', 'Owner', 'Score']].to_excel(
            writer, sheet_name='Weekly Highs', index=False
        )
        
        # Tab 3: All Scores
        df.sort_values(['Week', 'Score'], ascending=[True, False]).to_excel(
            writer, sheet_name='All Scores', index=False
        )
        
        # Tab 4: League Parameters
        params_df.to_excel(writer, sheet_name='Parameters', index=False)
    
    print(f"\n✅ Excel file created: {output_file}")
    print("\nTabs included:")
    print("  1. Final Payouts - Complete payout breakdown by owner")
    print("  2. Weekly Highs - Weekly high score winners")
    print("  3. All Scores - All team scores by week")
    print("  4. Parameters - League configuration and audit trail")
    print("\n📊 Upload this file directly to Google Sheets!")
    
except ImportError:
    print("\n⚠️  openpyxl not installed. Installing...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'openpyxl'])
    print("✓ Installed openpyxl. Please re-run this cell.")